# Notebook prerequisites

NOTICE: These are prerequisite steps to run the examples on Binder. Locally, you probably do not have to execute the cell below

This is to install PyStoG into the environment and then matplotlib for visualization.


In [ ]:
!type python
!python -m pip install ../
!python -m pip install matplotlib

# How to use the Transformer class from PyStoG

This tutorial shows how to use the `Transformer` class from `pystog`


In [ ]:
import os

import numpy as np

from pystog import Transformer

## Converting real space functions

To use the `transformer` class, we first need some data. Lets load in some test data for Argon from our test suite. 
We are going to extract the $r$ and $g(r)$ from small molecular dynamics simulations generated using [LAMMPS](https://lammps.sandia.gov/). 

In [ ]:
data_file = os.path.join("..", "tests", "test_data", "argon.gr")
i, r, gr, nr = np.loadtxt(data_file, unpack=True, skiprows=5)
print("r vector: ", r)

Let's look at the data that we imported:

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.plot(r, gr)
plt.ylabel("$g(r)$")
plt.xlabel(r"r [$\AA$]")
plt.show()

Okay, now we create an instance of a `transformer` object using the `Transformer` class:

In [ ]:
transformer = Transformer()

We need to pass in some material-specific information to the converter
functions. Here is a dictionary of the values we need. There is also a testing
utility class `Argon` with these values for the dataset found [here](https://github.com/neutrons/pystog/blob/master/tests/materials.py).

In [ ]:
kwargs = {"rho": 0.02138, "<b_coh>^2": 3.644, "<b_tot^2>": 5.435}

We also need to create the $Q$ vector domain we will map our Fourier transform to:

In [ ]:
buffer = 0.001
q = np.arange(0.12, 35.0 + buffer, 0.02)
print("Q vector: ", q)

We now have the functions available to convert our $g(r)$ function to reciprocal space functions:

In [ ]:
q, sq, dsq = transformer.g_to_S(r, gr, q, **kwargs)
q, fq, dfq = transformer.g_to_F(r, gr, q, **kwargs)
q, fq_keen, dfq_keen = transformer.g_to_FK(r, gr, q, **kwargs)
q, dcs, ddcs = transformer.g_to_DCS(r, gr, q, **kwargs)

Now we can plot the functions to compare:

In [ ]:
# Creates two subplots and unpacks the output array immediately
%matplotlib inline
import matplotlib.pyplot as plt

f, ax = plt.subplots(2, 2, sharex=True)
f.tight_layout()
xlabel = r"Q [$\AA^{-1}$]"

ax[0, 0].plot(q, sq)
ax[0, 0].set_ylabel("$S(Q)$")

ax[0, 1].plot(q, fq)
ax[0, 1].set_ylabel("$Q[S(Q)-1]$")
ax[1, 1].set_xlabel(xlabel)

ax[1, 0].plot(q, fq_keen)
ax[1, 0].set_ylabel("$F(Q)$ [Keen version]")
ax[1, 0].set_xlabel(xlabel)

ax[1, 1].plot(q, dcs)
ax[1, 1].set_ylabel(r"$d_{\sigma} / d_{\Omega}$")
ax[1, 1].set_xlabel(xlabel)
plt.show()

## Converting reciprocal space functions

In [ ]:
transformer = Transformer()

Now let us load in some reciprocal space test data for Argon from our test suite. 
We are going to extract the $Q$ and $S(Q)$. These are actually generated exactly how we did in the tutorial above:

In [ ]:
data_file = os.path.join("..", "tests", "test_data", "argon.reciprocal_space.dat")
q, sq = np.loadtxt(data_file, unpack=True, skiprows=2, usecols=(0, 1))
q = q[5:]
sq = sq[5:]

Add lets look at the data we imported:

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

plt.plot(q, sq)
plt.ylabel("$S(Q)$")
plt.xlabel(r"Q [$\AA^{-1}$]")
plt.show()

Again, we need to pass in some material-specific information to the converter
functions. We replicate the `kwargs` just for convenience here:

In [ ]:
kwargs = {"rho": 0.02138, "<b_coh>^2": 3.644, "<b_tot^2>": 5.435}

We also need to create the $r$ vector domain we will map our Fourier transform to:

In [ ]:
buffer = 0.001
r = np.arange(0.075, 63.975 + buffer, 0.05)
print("r vector: ", r)

In [ ]:
r, gr, dgr = transformer.S_to_g(q, sq, r, **kwargs)
r, GofR, dGofR = transformer.S_to_G(q, sq, r, **kwargs)
r, GKofR, dGKofR = transformer.S_to_GK(q, sq, r, **kwargs)

And let us plot the output data:

In [ ]:
# Creates two subplots and unpacks the output array immediately
%matplotlib inline
import matplotlib.pyplot as plt

f, ax = plt.subplots(3, 1, sharex=True)
ax[0].plot(r, gr)
ax[0].set_ylabel("g(r)")

ax[1].plot(r, GofR)
ax[1].set_ylabel("G(r)")

ax[2].plot(r, GKofR)
ax[2].set_ylabel("G(r) [Keen version]")
ax[2].set_xlabel(r"r [$\AA$]")
plt.show()